# Lab 6

You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against $n$ computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it. 

A few requirements:
* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16). 
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

In [ ]:
import random

class Card:
    suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
    ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
    
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit
        self.value = self.card_value()

    def card_value(self):
        """Returns the value of the card for blackjack."""
        if self.rank in ['J', 'Q', 'K']:
            return 10
        elif self.rank == 'A':
            return 11
        else:
            return int(self.rank)

    def __repr__(self):
        return f"{self.rank} of {self.suit}"

class Deck:
    def __init__(self, num_decks=6):
        """Initializes the deck with a specified number of 52-card sets."""
        self.num_decks = num_decks
        self.cards = self.create_deck()
        self.plastic_card_placed = False

    def create_deck(self):
        """Creates a deck with multiple 52-card sets."""
        deck = []
        for _ in range(self.num_decks):
            for suit in Card.suits:
                for rank in Card.ranks:
                    deck.append(Card(rank, suit))
        return deck

    def shuffle_deck(self):
        """Shuffles the deck. If a plastic card is placed, shuffle the deck again."""
        if not self.plastic_card_placed:
            # Insert a plastic card at a random position in the deck
            self.plastic_card_placed = True
            plastic_card = Card('Plastic', 'Plastic')
            self.cards.append(plastic_card)
        random.shuffle(self.cards)

    def draw_card(self):
        """Draws a card from the deck."""
        if not self.cards:
            raise ValueError("Deck is empty!")
        return self.cards.pop()

    def reset_deck(self):
        """Resets the deck (creates a new one and shuffles)."""
        self.cards = self.create_deck()
        self.plastic_card_placed = False
        self.shuffle_deck()

    def __repr__(self):
        return f"Deck with {len(self.cards)} cards"


In [ ]:

deck = Deck(num_decks=6)

# Shuffle the deck
deck.shuffle_deck()

# Draw a card
card = deck.draw_card()
print(f"Card drawn: {card}")

deck.reset_deck()


2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook. 

3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.

4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

In [ ]:
import random

# Hand class: holds the cards of a player or dealer
class Hand:
    def __init__(self):
        self.cards = []

    def add_card(self, card):
        self.cards.append(card)

    def calculate_total(self):
        total = sum(card.value for card in self.cards)
        num_aces = sum(1 for card in self.cards if card.rank == 'A')
        
        # Adjust for Aces
        while total > 21 and num_aces > 0:
            total -= 10  # Convert Ace from 11 to 1
            num_aces -= 1
        return total

    def is_busted(self):
        return self.calculate_total() > 21

    def __repr__(self):
        return f"Hand: {', '.join(map(str, self.cards))} Total: {self.calculate_total()}"

# Player class (general for human or computer)
class Player:
    def __init__(self, name, chips=1000):
        self.name = name
        self.chips = chips
        self.hand = Hand()

    def bet(self, amount):
        """Player bets chips."""
        if amount > self.chips:
            raise ValueError("Not enough chips to make that bet!")
        self.chips -= amount
        return amount

    def win(self, amount):
        """Player wins chips."""
        self.chips += amount

    def lose(self, amount):
        """Player loses chips."""
        self.chips -= amount

    def make_move(self):
        raise NotImplementedError("Subclasses should implement this method.")

# Human Player class
class HumanPlayer(Player):
    def __init__(self, name, chips=1000):
        super().__init__(name, chips)

    def make_move(self, deck):
        while True:
            print(self.hand)
            move = input("Do you want to hit or stand? (h/s): ").strip().lower()
            if move == 'h':
                self.hand.add_card(deck.draw_card())
                if self.hand.is_busted():
                    print(f"{self.name} has busted with {self.hand}.")
                    return False  # The player loses this round
            elif move == 's':
                print(f"{self.name} stands with {self.hand}.")
                return True  # The player stands
            else:
                print("Invalid choice. Please choose 'h' or 's'.")

# Dealer class
class Dealer(Player):
    def __init__(self, name="Dealer", chips=1000):
        super().__init__(name, chips)

    def make_move(self, deck):
        print(f"{self.name} is playing their hand.")
        while self.hand.calculate_total() < 17:
            print(f"{self.name} hits and draws {deck.draw_card()}.")
            self.hand.add_card(deck.draw_card())
            if self.hand.is_busted():
                print(f"{self.name} has busted with {self.hand}.")
                return False  # Dealer loses
        print(f"{self.name} stands with {self.hand}.")
        return True  # Dealer stands

# Game class
class Game:
    def __init__(self, player):
        self.deck = Deck(num_decks=6)
        self.dealer = Dealer()
        self.player = player

    def deal_cards(self):
        self.player.hand = Hand()
        self.dealer.hand = Hand()
        for _ in range(2):  # Deal 2 cards to each player
            self.player.hand.add_card(self.deck.draw_card())
            self.dealer.hand.add_card(self.deck.draw_card())

    def play_round(self):
        self.deck.shuffle_deck()
        self.deal_cards()

        # Player's turn
        if not self.player.make_move(self.deck):
            print(f"{self.player.name} has lost this round.")
            return

        # Dealer's turn
        if not self.dealer.make_move(self.deck):
            print(f"{self.dealer.name} has lost this round.")
            return

        # Determine the winner
        player_total = self.player.hand.calculate_total()
        dealer_total = self.dealer.hand.calculate_total()

        print(f"{self.player.name} has {player_total}, {self.dealer.name} has {dealer_total}.")

        if player_total > dealer_total and not self.player.hand.is_busted():
            print(f"{self.player.name} wins!")
            self.player.win(200)  # Example win
        elif dealer_total > player_total and not self.dealer.hand.is_busted():
            print(f"{self.dealer.name} wins!")
            self.player.lose(200)  # Example lose
        else:
            print("It's a tie!")

if __name__ == "__main__":
    human_player = HumanPlayer(name="Player1", chips=1000)
    game = Game(player=human_player)

    for _ in range(5):  # Play 5 rounds for testing
        game.play_round()


5.  Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.

In [ ]:
if __name__ == "__main__":
    # Create human player
    human_player = HumanPlayer(name="Player1")

    # Create game with 3 dealers
    game = Game(human_player=human_player, num_dealers=3)

    # Play 5 rounds of the game
    for _ in range(5):
        game.play_round()

6. Implement a new player with the following strategy:

    * Assign each card a value: 
        * Cards 2 to 6 are +1 
        * Cards 7 to 9 are 0 
        * Cards 10 through Ace are -1
    * Compute the sum of the values for all cards seen so far.
    * Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.  

In [ ]:
class CardCountingPlayer(Player):
    def __init__(self, name, threshold=-2):
        super().__init__(name)
        self.running_count = 0  # Start with a count of 0
        self.threshold = threshold  # Threshold for hit/stand decision

    def update_running_count(self, card):
        """ Update the running count based on the card value. """
        if card.rank in ['2', '3', '4', '5', '6']:
            self.running_count += 1
        elif card.rank in ['10', 'Jack', 'Queen', 'King', 'Ace']:
            self.running_count -= 1

    def make_move(self, deck):
        """ Make the move based on the running count. """
        print(f"{self.name}'s hand: {self.hand}")
        # The player hits if the running count is lower than the threshold
        while self.hand.calculate_total() < 21:
            if self.running_count < self.threshold:
                print(f"{self.name} hits.")
                card = deck.draw_card()
                self.hand.add_card(card)
                print(f"{self.name} draws {card}.")
                self.update_running_count(card)  # Update running count after drawing a card
            else:
                print(f"{self.name} stands with {self.hand}.")
                return True  # The player stands

        print(f"{self.name} has busted with {self.hand}.")
        return False  # Busts

if __name__ == "__main__":
    # Create human player and card counting player
    human_player = Player(name="Player1")
    card_counting_player = CardCountingPlayer(name="Card Counter", threshold=-2)

    # Create game with 3 dealers and the card counting player
    game = Game(human_player=human_player, num_dealers=3, card_counting_player=card_counting_player)

    # Play 5 rounds of the game
    for _ in range(5):
        game.play_round()


7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

In [ ]:
# Dealer class (inherits from Player)
class Dealer(Player):
    def __init__(self, name):
        super().__init__(name)

    def make_move(self, deck, verbose=True):
        if verbose:
            print(f"{self.name}'s hand: {self.hand}")
        while self.hand.calculate_total() < 17:
            card = deck.draw_card()
            self.hand.add_card(card)
            if verbose:
                print(f"{self.name} hits and draws {card}.")
            if self.hand.calculate_total() > 21:
                if verbose:
                    print(f"{self.name} has busted with {self.hand}.")
                return False  # Dealer busts
        if verbose:
            print(f"{self.name} stands with {self.hand}.")
        return True

# Game class
class Game:
    def __init__(self, human_player, num_dealers=3, card_counting_player=None, verbose=False):
        self.deck = Deck(num_decks=6)  # 6 decks in the game
        self.human_player = human_player
        self.dealers = [Dealer(name=f"Dealer{i+1}") for i in range(num_dealers)]
        self.card_counting_player = card_counting_player
        self.verbose = verbose  # Add verbose mode

    def deal_cards(self):
        # Reset hands for all players (including human player and dealers)
        self.human_player.hand = Hand()
        for dealer in self.dealers:
            dealer.hand = Hand()
        if self.card_counting_player:
            self.card_counting_player.hand = Hand()
        # Deal 2 cards to each player
        self.human_player.hand.add_card(self.deck.draw_card())
        self.human_player.hand.add_card(self.deck.draw_card())
        for dealer in self.dealers:
            dealer.hand.add_card(self.deck.draw_card())
            dealer.hand.add_card(self.deck.draw_card())
        if self.card_counting_player:
            self.card_counting_player.hand.add_card(self.deck.draw_card())
            self.card_counting_player.hand.add_card(self.deck.draw_card())

    def play_round(self):
        self.deck.shuffle_deck()
        self.deal_cards()

        if self.verbose:
            print("\n-- Round Start --")

        # Human player's turn
        if not self.human_player.make_move(self.deck, self.verbose):
            if self.verbose:
                print(f"{self.human_player.name} has busted.\n")
            return False
        
        # Card counting player's turn
        if self.card_counting_player:
            if not self.card_counting_player.make_move(self.deck, self.verbose):
                if self.verbose:
                    print(f"{self.card_counting_player.name} has busted.\n")
                return False
        
        # Dealers' turns
        for dealer in self.dealers:
            if not dealer.make_move(self.deck, self.verbose):
                if self.verbose:
                    print(f"{dealer.name} has busted.")
        
        # Determine the best outcome
        human_total = self.human_player.hand.calculate_total()
        if self.verbose:
            print(f"{self.human_player.name} has {human_total}.")

        for dealer in self.dealers:
            dealer_total = dealer.hand.calculate_total()
            if self.verbose:
                print(f"{dealer.name} has {dealer_total}.")
            if human_total > 21:
                if self.verbose:
                    print(f"{self.human_player.name} has busted. {dealer.name} wins!")
                self.human_player.lose(200)
            elif dealer_total > 21:
                if self.verbose:
                    print(f"{dealer.name} has busted. {self.human_player.name} wins!")
                self.human_player.win(200)
            elif human_total > dealer_total:
                if self.verbose:
                    print(f"{self.human_player.name} wins!")
                self.human_player.win(200)
            elif dealer_total > human_total:
                if self.verbose:
                    print(f"{dealer.name} wins!")
                self.human_player.lose(200)
            else:
                if self.verbose:
                    print("It's a tie!")

        if self.verbose:
            print("-- Round End --\n")
        return True

# Run the game with card counting player and dealer players
def run_simulation():
    # Create the card counting player
    card_counting_player = CardCountingPlayer(name="Card Counter", threshold=-2)
    human_player = Player(name="Regular Player")

    # Create 3 dealer players
    game = Game(human_player=human_player, num_dealers=3, card_counting_player=card_counting_player, verbose=False)

    # Play 50 rounds or until the card counting player is out of money
    rounds_played = 0
    while card_counting_player.chips > 0 and rounds_played < 50:
        game.play_round()
        rounds_played

8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?


In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np

# Constants
SUITS = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']
VALUES = {str(n): n for n in range(2, 11)}
VALUES.update({'Jack': 10, 'Queen': 10, 'King': 10, 'Ace': 11})

class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = VALUES[rank]

    def __repr__(self):
        return f"{self.rank} of {self.suit}"

class Deck:
    def __init__(self, num_decks=6):
        self.cards = [Card(suit, rank) for suit in SUITS for rank in RANKS] * num_decks
        random.shuffle(self.cards)

    def draw_card(self):
        return self.cards.pop() if self.cards else None

class Hand:
    def __init__(self):
        self.cards = []
        self.value = 0
        self.aces = 0

    def add_card(self, card):
        self.cards.append(card)
        self.value += card.value
        if card.rank == 'Ace':
            self.aces += 1
        self.adjust_for_aces()

    def adjust_for_aces(self):
        while self.value > 21 and self.aces:
            self.value -= 10
            self.aces -= 1

class Player:
    def __init__(self, name, strategy):
        self.name = name
        self.chips = 1000
        self.hand = Hand()
        self.strategy = strategy

    def play(self, dealer_card):
        return self.strategy(self.hand, dealer_card)

class Dealer:
    def __init__(self):
        self.hand = Hand()

    def play(self):
        while self.hand.value < 17:
            self.hand.add_card(deck.draw_card())

class Game:
    def __init__(self, players):
        self.deck = Deck()
        self.dealer = Dealer()
        self.players = players
        self.results = []

    def deal_initial_cards(self):
        for player in self.players:
            player.hand.add_card(self.deck.draw_card())
            player.hand.add_card(self.deck.draw_card())
        self.dealer.hand.add_card(self.deck.draw_card())
        self.dealer.hand.add_card(self.deck.draw_card())

    def play_round(self):
        self.deal_initial_cards()
        dealer_card = self.dealer.hand.cards[0]

        for player in self.players:
            while player.play(dealer_card):
                player.hand.add_card(self.deck.draw_card())
            print(f"{player.name}'s hand: {player.hand.cards} (Value: {player.hand.value})")

        self.dealer.play()
        print(f"Dealer's hand: {self.dealer.hand.cards} (Value: {self.dealer.hand.value})")

        self.evaluate_winner()

    def evaluate_winner(self):
        for player in self.players:
            if player.hand.value > 21:
                print(f"{player.name} busts!")
            elif self.dealer.hand.value > 21 or player.hand.value > self.dealer.hand.value:
                print(f"{player.name} wins!")
                player.chips += 100  # Win condition
            elif player.hand.value < self.dealer.hand.value:
                print(f"{player.name} loses.")
                player.chips -= 100  # Lose condition
            else:
                print(f"{player.name} pushes.")

def basic_strategy(hand, dealer_card):
    return hand.value < 17  # Basic strategy: hit below 17

def simulate_games(num_games=100, rounds_per_game=50):
    winnings_list = []

    for _ in range(num_games):
        players = [Player("Player 1", basic_strategy)]
        total_winnings = []
        
        for _ in range(rounds_per_game):
            game = Game(players)
            game.play_round()
            total_winnings.append(players[0].chips)
            game.deck = Deck()  # Reset deck for next round
        
        # Calculate winnings after 50 rounds
        winnings_list.append(total_winnings[-1] - 1000)  # Initial chips were 1000

    return winnings_list

# Run simulations
winnings = simulate_games(100, 50)

# Calculate average and standard deviation
average_winnings = np.mean(winnings)
std_deviation = np.std(winnings)

# Probability of net winning or losing
probability_winning = np.sum(np.array(winnings) > 0) / len(winnings)
probability_losing = np.sum(np.array(winnings) < 0) / len(winnings)

print(f"Average winnings per game: {average_winnings}")
print(f"Standard deviation of winnings: {std_deviation}")
print(f"Probability of winning: {probability_winning}")
print(f"Probability of losing: {probability_losing}")

plt.hist(winnings, bins=20, edgecolor='black')
plt.title('Histogram of Winnings after 50 Rounds')
plt.xlabel('Winnings ($)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?

In [ ]:
thresholds = range(15, 22)  # Test thresholds from 15 to 21
results = {}

for threshold in thresholds:
    winnings = simulate_games(100, 50, threshold)
    
    average_winnings = np.mean(winnings)
    std_deviation = np.std(winnings)
    probability_winning = np.sum(np.array(winnings) > 0) / len(winnings)
    probability_losing = np.sum(np.array(winnings) < 0) / len(winnings)

    results[threshold] = {
        'average_winnings': average_winnings,
        'std_deviation': std_deviation,
        'probability_winning': probability_winning,
        'probability_losing': probability_losing
    }

# Display results
for threshold, stats in results.items():
    print(f"Threshold: {threshold}")
    print(f"  Average winnings: {stats['average_winnings']:.2f}")
    print(f"  Standard deviation: {stats['std_deviation']:.2f}")
    print(f"  Probability of winning: {stats['probability_winning']:.2f}")
    print(f"  Probability of losing: {stats['probability_losing']:.2f}")
    print()

10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings. 

In [ ]:
def dynamic_threshold_strategy(hand, dealer_card):
    if 2 <= dealer_card.value <= 6:  # Dealer has a weak card
        return hand.value < 18  # Aggressive play
    else:  # Dealer has a strong card
        return hand.value < 16  # Conservative play

def simulate_games(num_games=100, rounds_per_game=50, strategy):
    winnings_list = []

    for _ in range(num_games):
        players = [Player("Player 1", strategy)]
        total_winnings = []
        
        for _ in range(rounds_per_game):
            game = Game(players)
            game.play_round()
            total_winnings.append(players[0].chips)
            game.deck = Deck()  # Reset deck for next round
        
        winnings_list.append(total_winnings[-1] - 1000)  # Calculate net winnings

    return winnings_list

# Compare strategies
num_games = 100
rounds_per_game = 50
)
basic_strategy = lambda hand, dealer_card: hand.value < 17
basic_winnings = simulate_games(num_games, rounds_per_game, basic_strategy)

# Dynamic Threshold Strategy
dynamic_winnings = simulate_games(num_games, rounds_per_game, dynamic_threshold_strategy)

def calculate_statistics(winnings):
    average = np.mean(winnings)
    std_dev = np.std(winnings)
    probability_winning = np.sum(np.array(winnings) > 0) / len(winnings)
    probability_losing = np.sum(np.array(winnings) < 0) / len(winnings)
    return average, std_dev, probability_winning, probability_losing

basic_stats = calculate_statistics(basic_winnings)
dynamic_stats = calculate_statistics(dynamic_winnings)

strategies = ['Basic Strategy', 'Dynamic Threshold Strategy']
results = [basic_stats, dynamic_stats]

for i, strategy in enumerate(strategies):
    print(f"{strategy}:")
    print(f"  Average winnings: {results[i][0]:.2f}")
    print(f"  Standard deviation: {results[i][1]:.2f}")
    print(f"  Probability of winning: {results[i][2]:.2f}")
    print(f"  Probability of losing: {results[i][3]:.2f}")
    print()

